In [2]:
import pandas as pd
from datetime import datetime, timedelta

# Load data from CSV file
df = pd.read_csv('Ajmer.csv')

# Print column names to check for 'datetime' column
print("Column names in the dataset:", df.columns)

# Convert 'datetime' column to datetime
date_column_name = 'datetime'
df[date_column_name] = pd.to_datetime(df[date_column_name])

def check_heatwave(input_date, df, date_column_name):
    input_date = pd.to_datetime(input_date)
    start_date = input_date - timedelta(days=2)
    end_date = input_date + timedelta(days=2)
    
    # Filter the DataFrame for the range around the input date
    df_filtered = df[(df[date_column_name] >= start_date) & (df[date_column_name] <= end_date)]
    
    if len(df_filtered) < 3:
        return "Not enough data to determine a heatwave"
    
    heatwave = False
    for i in range(len(df_filtered) - 2):
        if (df_filtered.iloc[i]['tempmax'] > 90 and
            df_filtered.iloc[i+1]['tempmax'] > 90 and
            df_filtered.iloc[i+2]['tempmax'] > 90):
            heatwave = True
            break
    
    return "Heatwave" if heatwave else "No heatwave"

# Get user input date
input_date = input("Enter the date (YYYY-MM-DD): ")
result = check_heatwave(input_date, df, date_column_name)
print(result)


Column names in the dataset: Index(['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations'],
      dtype='object')


No heatwave


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
# Load dataset
data = pd.read_csv('Ajmer.csv')

# Parse datetime
data['datetime'] = pd.to_datetime(data['datetime'])
data['year'] = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['day'] = data['datetime'].dt.day

# Handle missing values
data = data.fillna(method='ffill')

# Convert categorical data to numeric
data = pd.get_dummies(data, columns=['conditions', 'description', 'icon', 'preciptype', 'stations'])

# Feature and target variables
features = data.drop(columns=['temp', 'datetime'])
target = data['temp']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
def predict_future_temperature(model, data, start_date, days_ahead=5):
    start_date = pd.to_datetime(start_date)
    predictions = []
    
    for i in range(days_ahead):
        current_date = start_date + pd.Timedelta(days=i)
        feature_row = data[data['datetime'] == current_date].drop(columns=['temp', 'datetime'])
        if feature_row.empty:
            print(f"No data available for {current_date}")
            break
        prediction = model.predict(feature_row)
        predictions.append((current_date, prediction[0]))
    
    return predictions

# Predict temperatures for the next 5 days from a given date
start_date = '2024-06-15'
future_temperatures = predict_future_temperature(model, data, start_date)
for date, temp in future_temperatures:
    print(f'Temperature on {date.date()}: {temp:.2f}°C')


C:\Users\subbu\AppData\Local\Temp\ipykernel_11960\3611109750.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


ValueError: could not convert string to float: 'ajmer'